In [1]:
url = 'http://api.tvmaze.com/singlesearch/shows?q=westworld&embed=episodes'
import requests
import pandas as pd
import json
import csv
from datetime import datetime
from bs4 import BeautifulSoup
class episode:

    def __init__(self,url):
        self.url=url
    def data_download(self):
        response = requests.get(self.url)
        data = json.loads(response.text)
        data1=data['_embedded']['episodes']
        return data1
    def csv_file(self):
        data1=self.data_download()
        
        csv_file=open('df.csv','w')
        csv_writer=csv.writer(csv_file) #creating the object of the csv file
        count=0
        
        for i in data1:
            
            if count==0:
                header=i.keys()
                csv_writer.writerow(header)
                count+=1
            csv_writer.writerow(i.values())
        csv_file.close()
        df1=pd.read_csv('df.csv')
        return df1
    def date_time(self):
        df=self.csv_file()
        df['airtime'] = pd.to_datetime(df['airtime'])
        df['airtime'] = df['airtime'].dt.strftime("%I:%M %p")
        # Convert time string to time format in 12-hour format
        df['airtime'] = df['airtime'].apply(lambda x: datetime.strptime(x, '%I:%M %p').strftime('%I:%M %p'))
        df['airdate']=pd.to_datetime(df['airdate'])
        df['runtime']=df['runtime'].astype(float) # Changing the datatype to float
        # Getting the average rating from rating column
        for i in df['rating']:
            df['average rating']=eval(i)['average']
        # Creating a new column for medium & original image link
        for i in df['image']:
            a=eval(i)
            df['medium image link']=a['medium']
            df['Original image link']=a['original']
        
        return df
            
    def extract_summary(self):
        df=self.date_time()
        def extract_summary(html):
            
            soup = BeautifulSoup(html, 'html.parser')
            summary_tag = soup.find('p')
            if summary_tag:
                return summary_tag.get_text()
            else:
                return ""

        # Applying function to extract summaries
        df['summary'] = df['summary'].apply(extract_summary)
        df=df.drop(columns=['airstamp','rating','_links','image'])
        
        return df
url = 'http://api.tvmaze.com/singlesearch/shows?q=westworld&embed=episodes'
a=episode(url)
df=a.extract_summary()
df.dtypes


id                              int64
url                            object
name                           object
season                          int64
number                          int64
type                           object
airdate                datetime64[ns]
airtime                        object
runtime                       float64
summary                        object
average rating                float64
medium image link              object
Original image link            object
dtype: object